In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tqdm import tqdm
from pprint import pprint
%matplotlib inline


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split, cross_validate, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [2]:
df20t= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall.csv')

df20t['Team']

FileNotFoundError: [Errno 2] No such file or directory: './Data/2020_Team_DVOA_Ratings_Overall.csv'

In [ ]:
df20t1c= pd.read_csv('./Data/2020_Team_DVOA_Ratings_ Overall_ after_Week_1_convert.csv')
df20t1c.dtypes

In [ ]:

#df20t1c['TotalDVOAteam1']=df20t1c['TotalDVOAteam1'].astype('float64')
#df20t1c['TotalDVOAteam1']=pd.to_numeric(df20t1c['TotalDVOAteam1'], errors='coerce')
df20t1c['TotalDVOAteam1'].dtypes

In [ ]:
df20t1c['TotalDVOAteam1'].describe()

In [ ]:
df20t1= pd.read_csv('./Data/2020_Team_DVOA_Ratings_ Overall_ after_Week_1.csv')
df20t1b=df20t1
#df20t1b=df20t1b.reindex(['19', '3', '21', '3', '23', '17', '29', '8', '7', '25', '15', '22', '30', '24', '16']) #,"10", '14','5', '27', '0', '28', '2', '11, "4", "13", "9", "31", "18", "20", "6", "12", "26"])
df20t1b  

In [ ]:
df20t2= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_2.csv')
df20t2

In [ ]:
df20t2c= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_2 _convert.csv')
df20t2c.describe()

In [ ]:
df20t3= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_3.csv')
df20t3

In [ ]:
df20t3c= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_3_convert.csv')
df20t3c.columns

In [ ]:
df20t3c.describe()

In [ ]:
df20t4= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_4.csv')
df20t4b=df20t4
df20t4d=df20t4
df20t4b.describe()
# #put win loss
df20t4b=(df20t4b.drop(index=[4,16,23,30]))
df20t4b.describe()
df20t4b['W-L']=[0,0,1,0,1,
                1,1,0,0,1,
                1,0,1,0,1,
                0,1,0,1,1,
                1,1,1,0,0,
                0,0,0]
df20t4b.describe()
df20t4d=df20t4b
df20t4b.describe()
#Change column name
df20t4b=df20t4b.rename(columns={'Team':'Team', 'W-L':'W-L','Total DVOA Rank':'TotalDVOARankteam1', ' Total DVOA ':'TotalDVOAteam1','Weighted DVOA Rank':'WeightedDVOARankteam1','Weighted DVOA':'WeightedDVOAteam1', 'Offense DVOA Rank':'OffenseDVOARankteam1','Offense DVOA':'Offense DVOAteam1','Offense Weighted DVOA Rank':'OffenseWeightedDVOARankteam1','Offense Weighted DVOA':'OffenseWeightedDVOAteam1', 'Defense DVOA Rank':'DefenseDVOARankteam1','Defense DVOA':'DefenseDVOAteam1','Defense Weighted DVOA Rank':'DefenseWeightedDVOARankteam1', 'Defense Weighted DVOA':'DefenseWeightedDVOAteam1','Special Teams DVOA Rank':'SpecialTeamsDVOARankteam1', 'Special Teams DVOA': 'Special Teams DVOA','Special Teams Weighted DVOA Rank':'SpecialTeamsWeightedDVOARankteam1', 'Special Teams Weighted DVOA':'SpecialTeamsWeightedDVOAteam1'})
df20t4b.columns
df20t4b['Team 2']=['CHI', 'LV', 'CIN', 'CLE', 'MIN', 'PHI', 'LAC', 'MIA', 'TEN', 'WAS', 'IND', 'SEA', 'BUF', 'HOU', 'NYG', 'NO', 'ATL', 'BAL', 'JAX', 'TB', 'NYJ', 'KC', 'SF', 'CAR', 'LAR', 'ARI', 'PIT', 'DAL']
                                 
# ''Team', 'W-L', 'Total DVOA Rank', 'Total DVOA', 'Weighted DVOA Rank',
#        'Weighted DVOA', 'Offense DVOA Rank', 'Offense DVOA',
#        'Offense Weighted DVOA Rank', 'Offense Weighted DVOA',
#        'Defense DVOA Rank', 'Defense DVOA', 'Defense Weighted DVOA Rank',
#        'Defense Weighted DVOA',
#                                  'Special Teams DVOA Rank',
#                                    'Special Teams DVOA', 
#                                  'Special Teams Weighted DVOA Rank',
#                                'Special Teams Weighted DVOA'                               
                                
#                                  'Team', 'W-L', 'TotalDVOARankteam1', 'TotalDVOAteam1',
#        'WeightedDVOARankteam1', 'WeightedDVOAteam1', 'OffenseDVOARankteam1',
#        'Offense DVOAteam1', 'OffenseWeightedDVOARankteam1',
#        'OffenseWeightedDVOAteam1', 'DefenseDVOARankteam1', 'DefenseDVOAteam1',
#        'DefenseWeightedDVOARankteam1', 'DefenseWeightedDVOAteam1',
#        'SpecialTeamsDVOARankteam1', 'Special Teams DVOA',
#        'SpecialTeamsWeightedDVOARankteam1', 'SpecialTeamsWeightedDVOAteam1'
#change column name
df20t4d['Home 1st team']=[0,1,1,0,1,1,1,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,1,1,0,0]
#change win loss
df20t4d=df20t4d.rename(columns={'W-L':'W-L.1', 'Special Teams DVOA':'Special Teams DVOA.1', 'Team':'Team 2'})
#df20t4d['Team 2']=['CHI', 'LV', 'CIN', 'CLE', 'MIN', 'PHI', 'LAC', 'MIA', 'TEN', 'WAS', 'IND', 'SEA', 'BUF', 'HOU', 'NYG', 'NO', 'ATL', 'BAL', 'JAX', 'TB', 'NYJ', 'KC', 'SF', 'CAR', 'LAR', 'ARI', 'PIT', 'DAL']
df20t4d['W-L.1']=[1,1,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,1,1,1,1]
#add home field
#merge on  opponent team
df20t4c=pd.merge(df20t4b, df20t4d, how='outer' ,on=['Team 2'])

df20t4c.columns


In [ ]:
df20t5= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_5.csv')
df20t5

In [ ]:
df20t6= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_6.csv')
df20t6.columns

In [ ]:
df20t7= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_7.csv')
df20t7

In [ ]:
df20t8= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_8.csv')
df20t8

In [ ]:
df20t9= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_9.csv')
df20t9

In [ ]:
df20t10= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_10.csv')
df20t10

In [ ]:
df20t11= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_11.csv')
df20t11

In [ ]:
df20t12= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_12.csv')
df20t12

In [ ]:
df20t13= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_13.csv')
df20t13

In [ ]:
df20t14= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_14.csv')
df20t14

In [ ]:
df20t15= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_15.csv')
df20t15

In [ ]:
df20t16= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_16.csv')
df20t16

In [ ]:
df20t17= pd.read_csv('./Data/2020_Team_DVOA_Ratings_Overall_after_Week_17.csv')


In [ ]:
combine = [df20t1, df20t2, df20t3, df20t4, df20t5, df20t6, df20t7, df20t8, df20t9, df20t10, df20t11, df20t12,\
           df20t13, df20t14, df20t15, df20t16, df20t17]
dfactual=pd.concat(combine)
dfactual

In [ ]:
combineprojected=[df20t1c, df20t2c, df20t3c, df20t4c]

In [ ]:
dfprojected=pd.concat(combineprojected)
dfprojected

In [ ]:
dfprojected.columns

In [ ]:
# features=['TotalDVOAteam1', 'WeightedDVOAteam1','Offense DVOAteam1', 'OffenseWeightedDVOAteam1', 'DefenseDVOAteam1','DefenseWeightedDVOAteam1', 'Special Teams DVOA', 'SpecialTeamsWeightedDVOAteam1', 'Total DVOA',
# 'Weighted DVOA Rank', 'Weighted DVOA', 'Offense DVOA Rank','Offense DVOA', 'Offense Weighted DVOA', 'Defense DVOA','Defense Weighted DVOA', 'Special Teams DVOA.1',
# 'Special Teams Weighted DVOA', 'Home 1st team']
features=['OffenseWeightedDVOAteam1', 'DefenseWeightedDVOAteam1', 'SpecialTeamsWeightedDVOAteam1', 'Offense Weighted DVOA', 'Defense Weighted DVOA', 'Special Teams Weighted DVOA', 'Home 1st team']
target='W-L'

In [ ]:
X=dfprojected[features]
y=dfprojected[target]

In [ ]:
#instantiation
lr=LogisticRegression()
knn= KNeighborsClassifier(n_neighbors=1, weights='distance', metric='euclidean')
rf=RandomForestClassifier(n_estimators=25, n_jobs=-1)

In [ ]:
#Cross val scores
lr_scores = cross_val_score(lr, X, y, cv=KFold(n_splits=7, shuffle=True, random_state=42))
knn_scores=cross_val_score(knn, X, y, cv=KFold(n_splits=7, shuffle=True, random_state=42))
rf_scores=cross_val_score(rf, X, y, cv=KFold(n_splits=7, shuffle=True, random_state=42))

print('logreg', round (lr_scores.mean(), 3), '+-', round (2 *lr_scores.std(), 3))
print('KNN', round (knn_scores.mean(), 3), '+-', round (2 *lr_scores.std(), 3))
print('RF', round (rf_scores.mean(), 3), '+-', round (2 *lr_scores.std(), 3))
#Sophie Tabac's lesson nlp modeling walkthrough

In [ ]:
lrpredictions= cross_val_predict(lr, X, y, cv=KFold(n_splits=10, shuffle=True, random_state=42))
knnpredictions= cross_val_predict(knn, X, y, cv=KFold(n_splits=10, shuffle=True, random_state=42))
rfpredictions= cross_val_predict(rf, X, y, cv=KFold(n_splits=10, shuffle=True, random_state=42))

In [ ]:
cnf_matrix_lr=metrics.confusion_matrix(y, lrpredictions)
cnf_matrix_lr
#Riley Dallas's Classification MetricII lecture

In [ ]:
# name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(y))
plt.xticks(tick_marks, y)
plt.yticks(tick_marks, y)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix_lr), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion Matrix for Logistic Regression', y=1.1)
plt.ylabel('Actual Label')
plt.xlabel('Predicted label')
#Riley Dallas's Classification MetricII lecture

In [ ]:
print("Accuracy:",metrics.accuracy_score(y, lrpredictions))
print("Precision:",metrics.precision_score(y, lrpredictions))
print("Recall:",metrics.recall_score(y, lrpredictions))
print("F1 Score:", metrics.f1_score(y, lrpredictions))

In [ ]:
cnf_matrix_knn=metrics.confusion_matrix(y, knnpredictions)
cnf_matrix_knn
#Riley Dallas's Classification MetricII lecture

In [ ]:
# name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(y))
plt.xticks(tick_marks, y)
plt.yticks(tick_marks, y)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix_knn), annot=True, cmap="coolwarm" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix for KNN Model', y=1.1)
plt.ylabel('Actual Label')
plt.xlabel('Predicted label')
#Riley Dallas's Classification MetricII lecture

In [ ]:
print("Accuracy:",metrics.accuracy_score(y, knnpredictions))
print("Precision:",metrics.precision_score(y, knnpredictions))
print("Recall:",metrics.recall_score(y, knnpredictions))
print("F1 Score:", metrics.f1_score(y, knnpredictions))

In [ ]:
cnf_matrix_rf=metrics.confusion_matrix(y, rfpredictions)
cnf_matrix_rf
#Riley Dallas's Classification MetricII lecture

In [ ]:
# name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(y))
plt.xticks(tick_marks, y)
plt.yticks(tick_marks, y)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix_rf), annot=True, cmap="viridis" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix for Random Forests', y=1.1)
plt.ylabel('Actual')
plt.xlabel('Predicted')
#Riley Dallas's Classification MetricII lecture

In [ ]:
print("Accuracy:",metrics.accuracy_score(y, rfpredictions))
print("Precision:",metrics.precision_score(y, rfpredictions))
print("Recall:",metrics.recall_score(y, rfpredictions))
print("F1 Score:", metrics.f1_score(y, rfpredictions))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=33)

In [ ]:
X.shape

In [ ]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [ ]:
model = Sequential()
model.add(Dense(16,
                activation='relu',
                input_shape=(7,)))

#model will predict probability betwee .0 to 1
#1 neuron, sigmoid turns into probability
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# with accuracy as a metric
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy']) # optional param, great for classification# with accuracy as a metric


In [ ]:
res= model.fit(X_train, y_train, epochs=100,
              batch_size=512,
              validation_data=(X_test_sc, y_test),
              verbose=0)#0 means don't print

In [ ]:
res.history.keys()


In [ ]:
plt.plot(res.history['loss'], label='Train')
plt.plot(res.history['val_loss'], label='Test')
plt.legend();

In [ ]:
dftest2021WK1= pd.read_csv('./Data/2021_Week1.test.2020endinputs.csv')

In [ ]:
# features=['TotalDVOAteam1', 'WeightedDVOAteam1','Offense DVOAteam1', 'OffenseWeightedDVOAteam1', 'DefenseDVOAteam1',
# 'DefenseWeightedDVOAteam1', 'Special Teams DVOA', 'SpecialTeamsWeightedDVOAteam1', 'Total DVOA',
# 'Weighted DVOA Rank', 'Weighted DVOA', 'Offense DVOA Rank',
# 'Offense DVOA', 'Offense Weighted DVOA', 'Defense DVOA',
# 'Defense Weighted DVOA', 'Special Teams DVOA.1',
# 'Special Teams Weighted DVOA', 'Home 1st team']
features=['OffenseWeightedDVOAteam1', 'DefenseWeightedDVOAteam1', 'SpecialTeamsWeightedDVOAteam1', 'Offense Weighted DVOA', 'Defense Weighted DVOA', 'Special Teams Weighted DVOA', 'Home 1st team']

target='W-L'

Xvalidation1=dftest2021WK1[features]
#No y as the first of NFL starts-predictions made prior to games

In [ ]:

lr.fit(X, y)
knn.fit(X,y)
rf.fit(X,y)
ss = StandardScaler()
Xvalidation1_sc = ss.fit_transform(Xvalidation1)

In [ ]:
lrpred1 = lr.predict(Xvalidation1)
knnpred1 = knn.predict(Xvalidation1)
rfpred1 = rf.predict(Xvalidation1)
nnpred1= model.predict(Xvalidation1)

In [ ]:
dftest2021WK1.columns

In [ ]:


dfbet=dftest2021WK1[['Team']]
dfbet['LR Predictions']= lrpred1
dfbet['KNN Predictions']=knnpred1
dfbet['RF Predictions']=rfpred1
dfbet['NN Predictions']=np.round(nnpred1)
dfbet['Other Team']=dftest2021WK1[['Team 2']]
dfbet